In [ ]:
from google.colab import drive
import json
import numpy as np
import pandas as pd
import tensorflow as tf
import spacy
from collections import Counter
import torch
import torchtext
import torch.nn as nn
import torch.optim as optim
import random

In [ ]:
!pip install torchtext==0.6.0 --quiet

from torchtext.data import Field, BucketIterator

In [ ]:
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
# Read JSON data from file
with open('/gdrive/MyDrive/train_data1.json', 'r') as file:
    data = json.load(file)

In [ ]:
# Read JSON data from file
with open('/gdrive/MyDrive/test_data1_final.json', 'r') as file:
    test_data = json.load(file)

In [ ]:
import re

# Define a function to check if a sentence contains English words
def contains_english_words(sentence):
    return bool(re.search(r'[a-zA-Z]', sentence))

In [ ]:
import string

# Define the remove_punctuations function
def remove_punctuations(sentence):
    punctuations = list(string.punctuation)
    punctuations.append('।')
    punctuations.append('৷')
    punctuations.append('’')
    punctuations.append('‘')
    cleaned = ""
    for letter in sentence:
        if letter not in punctuations:
            cleaned += letter
    return cleaned


In [ ]:
# Process JSON data
source_sentences_train = []
target_sentences_train = []
len_tamil = []
len_english = []

id_train = []

for language_pair, language_data in data.items():
    if(language_pair == "English-Tamil"):
      print(f"Language Pair: {language_pair}")
      for data_type, data_entries in language_data.items():
          print(f"  Data Type: {data_type}")
          for entry_id, entry_data in data_entries.items():
              source = entry_data["source"].lower()
              target = entry_data["target"].lower()
              source = remove_punctuations(source)
              target = remove_punctuations(target)
              if not contains_english_words(target):
                source_sentences_train.append(source)
                target_sentences_train.append(target)
                id_train.append(entry_id)
                len_tamil.append(len(source.split(' ')))
                len_english.append(len(target.split(' ')))

Language Pair: English-Tamil
  Data Type: Train


In [ ]:
# Calculate the maximum length of tamil and English sentences
max_length_tamil = max(len_tamil)
max_length_english = max(len_english)

print(f"Maximum Length of tamil Sentence: {max_length_tamil} words")
print(f"Maximum Length of English Sentence: {max_length_english} words")

Maximum Length of tamil Sentence: 80 words
Maximum Length of English Sentence: 72 words


In [ ]:
len(target_sentences_train)

57065

In [ ]:
test_source = []
valid_id = []
len_test=[]


for language_pair, language_data in test_data.items():
    if(language_pair == "English-Tamil"):
      print(f"Language Pair: {language_pair}")
      for data_type, data_entries in language_data.items():
          print(f"  Data Type: {data_type}")
          for entry_id, entry_data in data_entries.items():
              source = entry_data["source"].lower()
              source = remove_punctuations(source)
              test_source.append(source)
              valid_id.append(entry_id)
              len_test.append(len(source.split(' ')))

Language Pair: English-Tamil
  Data Type: Test


In [ ]:
len(test_source)

16675

In [ ]:
print(max(len_test))

107


In [ ]:
nlp_en = spacy.load('en_core_web_sm')
# Tokenize English sentences
tokenized_source_sentences = []
for sentence in source_sentences_train:
    doc = nlp_en(sentence)
    tokens = [token.text for token in doc]
    tokenized_source_sentences.append(tokens)

In [ ]:
tokenized_test_source = []
for sentence in test_source:
    doc = nlp_en(sentence)
    tokens = [token.text for token in doc]
    tokenized_test_source.append(tokens)

In [ ]:
!pip install nltk

In [ ]:
pip install indic-nlp-library

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 15.7 MB/s eta 0:00:00


In [ ]:
import nltk

nltk.download("indic_nltk")

from indicnlp.tokenize import indic_tokenize


# Tokenize tamil sentences
tokenized_target_sentences = []
for sentence in target_sentences_train:
    tokens = indic_tokenize.trivial_tokenize(sentence)
    tokenized_target_sentences.append(tokens)

[nltk_data] Error loading indic_nltk: Package 'indic_nltk' not found
[nltk_data]     in index


In [ ]:
from torchtext.data import Dataset, Example, Field


def tokenize_english(text):
    return text

def tokenize_tamil(text):
    return text

class CustomTranslationDataset(Dataset):
    def __init__(self, source_sentences, target_sentences, source_field, target_field):
        fields = [('source', source_field), ('target', target_field)]
        examples = []
        for src, tgt in zip(source_sentences, target_sentences):
            examples.append(Example.fromlist([src, tgt], fields))
        super().__init__(examples, fields)
# Create Fields for English (source) and tamil (target) text
english = Field(tokenize=tokenize_english,
                lower=True,
                init_token="<sos>",
                eos_token="<eos>")

tamil = Field(tokenize=tokenize_tamil,
              lower=True,
              init_token="<sos>",
              eos_token="<eos>")


train_dataset = CustomTranslationDataset(tokenized_source_sentences, tokenized_target_sentences, english, tamil)


english.build_vocab(train_dataset,max_size=30000, min_freq=3)
tamil.build_vocab(train_dataset,max_size=30000, min_freq=3)



In [ ]:
print(len(english.vocab), len(tamil.vocab))

17573 27719


In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 32

train_iterator = BucketIterator(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    sort_within_batch=True,
    sort_key=lambda x: len(x.source),
    device=device
)


In [ ]:
count = 0
for data in train_iterator:
  if count < 1 :
    print("Shapes", data.source.shape, data.target.shape)
    print()
    print("English - ",*data.source, " Length - ", len(data.source))
    print()
    print("tamil - ",*data.target, " Length - ", len(data.target))
    temp_eng = data.source
    temp_tamil = data.target
    count += 1


Shapes torch.Size([18, 32]) torch.Size([18, 32])

English -  tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0') tensor([ 3187,  4126,  4054,  2420,     4,     4,     7,     7,    17,     4,
          466,    44,    11,     7,     4,     4,    41, 17208,    11,   186,
           46,     0, 14793,    33,    46,    68,   112,  3106,  4203,    41,
            4,     4], device='cuda:0') tensor([ 836, 1765,   10, 6884, 2037,  179,  522,    4, 8947, 1458,    4,  246,
          90,   15,    0, 1864,   29,    6, 4856,    8,  198, 1941,  180, 2594,
          28, 6870,   40,    4,    7, 2981, 2039, 5134], device='cuda:0') tensor([  11,  426, 1852,   34,  211,  427,  218, 4541,    7,    8,    0,    9,
         564,  152,  458,   10,    8,  187,    5, 3135,   63,    8,   21, 1879,
         197, 6705,  432,  161,  348,    8,  969,    5], device='cuda:0') tensor([   6,    4,    5,   56,   18,   12, 1305,  343,  142, 1176,

In [ ]:
temp_eng_idx = (temp_eng).cpu().detach().numpy()
temp_tamil_idx = (temp_tamil).cpu().detach().numpy()

In [ ]:
print(english.vocab.__dict__.keys())
print(list(english.vocab.__dict__.values()))
e = list(english.vocab.__dict__.values())
for i in e:
  print(i)

dict_keys(['freqs', 'itos', 'unk_index', 'stoi', 'vectors'])
[Counter({'the': 62959, 'of': 39130, 'and': 25035, 'in': 24856, 'is': 19500, 'to': 18168, ' ': 16480, 'a': 15372, 'for': 7741, 'are': 7426, 'on': 6739, 'this': 6484, 'from': 6304, 'it': 6127, 'with': 6027, 'that': 5406, 'by': 5280, 'be': 5051, 'as': 4283, 'was': 3966, 'you': 3843, 'at': 3718, 'also': 3616, 'or': 3541, 'can': 3538, 'there': 3318, 'which': 3185, 'has': 3139, 'not': 3008, 'an': 2841, 'have': 2747, 'one': 2472, 'its': 2354, 'all': 1965, 'after': 1933, 'will': 1873, 'should': 1866, 'if': 1857, 'but': 1837, 'india': 1789, 'i': 1746, 'their': 1681, 'what': 1670, 'he': 1670, 'his': 1658, 'time': 1631, 'more': 1625, 'your': 1600, 'then': 1574, 'these': 1564, 'some': 1512, 'other': 1511, 'been': 1498, 'they': 1473, 'like': 1441, 'me': 1421, 'water': 1412, 'do': 1408, 'many': 1394, 'only': 1392, 'my': 1368, 'people': 1326, 'were': 1312, 'most': 1293, 'when': 1265, 'temple': 1232, 'about': 1210, 'such': 1210, 'during': 1

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:

word_2_idx = dict(e[3])
idx_2_word = {}
for k,v in word_2_idx.items():
  idx_2_word[v] = k


In [ ]:
class EncoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
    super(EncoderLSTM, self).__init__()

    self.hidden_size = hidden_size

    # Number of layers in the lstm
    self.num_layers = num_layers

    # Regularization parameter
    self.dropout = nn.Dropout(p)
    self.tag = True

    self.embedding = nn.Embedding(input_size, embedding_size)

    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)

  def forward(self, x):
    embedding = self.dropout(self.embedding(x))
    outputs, (hidden_state, cell_state) = self.LSTM(embedding)

    return hidden_state, cell_state


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_size_encoder = len(english.vocab)
encoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
encoder_dropout = 0.5

encoder_lstm = EncoderLSTM(input_size_encoder, encoder_embedding_size,
                           hidden_size, num_layers, encoder_dropout).to(device)
print(encoder_lstm)

EncoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(17573, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
)


In [ ]:
class DecoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p, output_size):
    super(DecoderLSTM, self).__init__()
    self.hidden_size = hidden_size

    # Number of layers in the lstm
    self.num_layers = num_layers

    self.output_size = output_size

    # Regularization parameter
    self.dropout = nn.Dropout(p)

    self.embedding = nn.Embedding(input_size, embedding_size)

    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)

    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, x, hidden_state, cell_state):

    x = x.unsqueeze(0)
    embedding = self.dropout(self.embedding(x))

    outputs, (hidden_state, cell_state) = self.LSTM(embedding, (hidden_state, cell_state))
    predictions = self.fc(outputs)
    predictions = predictions.squeeze(0)

    return predictions, hidden_state, cell_state

In [ ]:
input_size_decoder = len(tamil.vocab)
decoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
decoder_dropout = 0.5
output_size = len(tamil.vocab)

decoder_lstm = DecoderLSTM(input_size_decoder, decoder_embedding_size,
                           hidden_size, num_layers, decoder_dropout, output_size).to(device)
print(decoder_lstm)

DecoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(27719, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  (fc): Linear(in_features=1024, out_features=27719, bias=True)
)


In [ ]:

for batch in train_iterator:
  print(batch.source.shape)
  print(batch.target.shape)
  break

x = batch.target[1]
print(x)

torch.Size([14, 32])
torch.Size([16, 32])
tensor([   39,  5999,     6, 21352,   125,   119, 14620,    10,   254,   107,
          416, 16228,     0,  3425,   223,  1310,  6064,   486,     0,   591,
          923, 13767,   731,    10,  5060,  1647,   454,  8196,    39, 12097,
            0,  3689], device='cuda:0')


In [ ]:

class Seq2Seq(nn.Module):
  def __init__(self, Encoder_LSTM, Decoder_LSTM):
    super(Seq2Seq, self).__init__()
    self.Encoder_LSTM = Encoder_LSTM
    self.Decoder_LSTM = Decoder_LSTM

  def forward(self, source, target, tfr=0.5):
    batch_size = source.shape[1]
    target_len = target.shape[0]
    target_vocab_size = len(tamil.vocab)

    outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

    hidden_state, cell_state = self.Encoder_LSTM(source)

    x = target[0]

    for i in range(1, target_len):
      output, hidden_state, cell_state = self.Decoder_LSTM(x, hidden_state, cell_state)
      outputs[i] = output
      best_guess = output.argmax(1)
      x = target[i] if random.random() < tfr else best_guess

    return outputs

In [ ]:

# Hyperparameters

learning_rate = 0.001
step = 0

model = Seq2Seq(encoder_lstm, decoder_lstm).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

pad_idx = tamil.vocab.stoi[""]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [ ]:
model

Seq2Seq(
  (Encoder_LSTM): EncoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(17573, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  )
  (Decoder_LSTM): DecoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(27719, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=1024, out_features=27719, bias=True)
  )
)

In [ ]:
def translate_sentence(model, sentence, english, tamil, device, max_length=107):
    nlp_en = spacy.load('en_core_web_sm')

    if type(sentence) == str:
        tokens = [token.text for token in nlp_en(sentence)]
    else:
        tokens = [token.lower() for token in sentence]
    tokens.insert(0, english.init_token)
    tokens.append(english.eos_token)
    text_to_indices = [english.vocab.stoi[token] for token in tokens]
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    # Build encoder hidden, cell state
    with torch.no_grad():
        hidden, cell = model.Encoder_LSTM(sentence_tensor)

    outputs = [tamil.vocab.stoi[""]]

    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.Decoder_LSTM(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)

        if output.argmax(1).item() == tamil.vocab.stoi[""]:
            break

    translated_sentence = [tamil.vocab.itos[idx] for idx in outputs]
    return translated_sentence[1:]



In [ ]:
def pred_trans(model, tokens, english, tamil, device, max_length=107):
    tokens.insert(0, english.init_token)
    tokens.append(english.eos_token)
    text_to_indices = [english.vocab.stoi[token] for token in tokens]
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    # Build encoder hidden, cell state
    with torch.no_grad():
        hidden, cell = model.Encoder_LSTM(sentence_tensor)

    outputs = [tamil.vocab.stoi[""]]

    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.Decoder_LSTM(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)

        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == tamil.vocab.stoi[""]:
            break

    translated_sentence = [tamil.vocab.itos[idx] for idx in outputs]
    return translated_sentence[1:]

In [ ]:
import time
from tqdm import tqdm
epoch_loss = 0.0
num_epochs = 10
sentence1 = "you may be able to find the information you need on the city government website"


for epoch in range(num_epochs):
  start_time = time.time()
  print("Epoch - {} / {}".format(epoch+1, num_epochs))
  model.eval()
  translated_sentence1 = translate_sentence(model, sentence1, english, tamil,device, max_length=107)
  print(f"Translated example sentence 1: \n {translated_sentence1}")


  model.train(True)
  for batch_idx, batch in tqdm(enumerate(train_iterator), total=len(train_iterator)):
    input = batch.source.to(device)
    target = batch.target.to(device)

    output = model(input, target)
    output = output[1:].reshape(-1, output.shape[2])
    target = target[1:].reshape(-1)


    optimizer.zero_grad()

    # Calculate the loss value for every epoch
    loss = criterion(output, target)

    # Calculate the gradients for weights & biases using back-propagation
    loss.backward()

    # Clip the gradient value is it exceeds > 1
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

    # Update the weights values using the gradients we calculated using bp
    optimizer.step()
    step += 1
    epoch_loss += loss.item()

  # Calculate the time taken for the epoch
  end_time = time.time()
  epoch_time = end_time - start_time
  print(f"Time taken for epoch {epoch + 1}: {epoch_time:.2f} seconds")

  print("Epoch_Loss - {}".format(loss.item()))
  print()

print(epoch_loss / len(train_iterator))


Epoch - 1 / 10
Translated example sentence 1: 
 ['லோயர்', 'வங்காளத்தை', 'சமைக்கப்படாத', '81', 'அவென்யூ', 'கோவிலும்', 'அருகில்', 'குள்ளநரி', 'அவர்களால்', 'நினைவுபடுத்தும்', 'டாலருக்கும்', 'சிரமத்துடன்', 'உடன்படிக்கை', 'உடன்படிக்கை', 'ஆட்சிக்குப்', 'புகை', 'கர்ப்பகாலம்', 'மாகாணமான', 'மார்ச்சின்', 'தொடுகிறது', 'மார்ச்சின்', 'தொடுகிறது', 'பெயரிடப்பட்ட', 'கொள்ளையடிக்கிறது', 'பலவீனமாகி', 'மார்ச்சின்', 'செழுமையான', 'அழுக', 'அழுக', 'அழுக', 'ஸ்டேஷன்', 'நடைமுறையில்', 'பாலத்தைக்', 'உரிமைகளின்', 'ஒன்றுமில்லை', 'உரிமைகளின்', 'படை', 'கர்ப்பகாலம்', 'நகரங்களில்', 'நகரங்களில்', 'லோசன்', 'லோசன்', 'கார்பரேஷன்', 'கார்பரேஷன்', 'கார்பரேஷன்', 'கிடங்குகளின்', 'லட்சத்தைச்', 'சேலம்', 'கொடுக்கப்பட்ட', 'வசதிக்கேற்ப', 'சேனலுக்கு', 'சரியாக', 'சரியாக', 'சரியாக', 'ராஜஸ்தானிய', 'தயாரிப்பதில்', 'தயாரிப்பதில்', 'த்வாரகா', 'இரண்டை', 'நெடுந்தூரம்', 'நெடுந்தூரம்', 'மேலேயுள்ள', 'குளிப்பது', 'டென்', 'மராட்டியர்கள்', 'லட்சத்தைச்', 'லட்சத்தைச்', 'கர்ப்பகாலம்', 'ஜுங்', 'மார்ச்சின்', 'நீதிபதியின்', 'நீதிபதியின்', 'பொருளாதார', 

100%|██████████| 1784/1784 [07:08<00:00,  4.16it/s]


Time taken for epoch 1: 431.85 seconds
Epoch_Loss - 5.971072196960449

Epoch - 2 / 10
Translated example sentence 1: 
 ['இந்த', 'ஒரு', 'ஒரு', 'ஒரு', 'ஒரு', 'ஒரு', '<eos>', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', 

100%|██████████| 1784/1784 [07:08<00:00,  4.17it/s]


Time taken for epoch 2: 428.71 seconds
Epoch_Loss - 4.703619480133057

Epoch - 3 / 10
Translated example sentence 1: 
 ['நீங்கள்', 'நீங்கள்', 'நீங்கள்', 'நீங்கள்', 'நீங்கள்', 'நீங்கள்', 'நீங்கள்', 'நீங்கள்', '<eos>', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', 

...

Time taken for epoch 8: 431.43 seconds
Epoch_Loss - 4.342070579528809

Epoch - 9 / 10
Translated example sentence 1: 
 ['நீங்கள்', 'நீங்கள்', 'நீங்கள்', 'நீங்கள்', 'நீங்கள்', 'நீங்கள்', 'நீங்கள்', 'நீங்கள்', 'நீங்கள்', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>'

100%|██████████| 1784/1784 [07:07<00:00,  4.17it/s]


Time taken for epoch 9: 428.50 seconds
Epoch_Loss - 2.2775557041168213

Epoch - 10 / 10
Translated example sentence 1: 
 ['நீங்கள்', 'நீங்கள்', 'நீங்கள்', 'நீங்கள்', 'நீங்கள்', 'நீங்கள்', 'நீங்கள்', 'நீங்கள்', 'நீங்கள்', 'நீங்கள்', '<eos>', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<p

100%|██████████| 1784/1784 [07:07<00:00,  4.17it/s]

Time taken for epoch 10: 428.46 seconds
Epoch_Loss - 2.7380824089050293

37.35049841593066


In [ ]:
s = test_source[10]
translation = translate_sentence(model, s, english, tamil, device, max_length=128)
cleaned_tokens = [token for token in translation if token not in ['<eos>', '<pad>']]
readable_translation = ' '.join(cleaned_tokens)
print(readable_translation)

மிக உயர்ந்த ஆறுகள் பன்றி மற்றும் மற்றும் மற்றும் மற்றும் மற்றும் மற்றும் மற்றும் மற்றும் மற்றும் மற்றும் மற்றும் மற்றும் மற்றும் மற்றும்


In [ ]:
tamil_pred = []

for tokens in tokenized_test_source:
  translation = pred_trans(model, tokens, english, tamil, device, max_length=107)
  cleaned_tokens = [token for token in translation if token not in ['<eos>', '<pad>','।','৷']]
  pred = ' '.join(cleaned_tokens)
  print(pred)
  tamil_pred.append(pred)


Streaming output truncated to the last 5000 lines.
மஞ்சளில் முறையில் பின்பு நீரிழிவுநோய் பின்பு பின்பு பின்பு கொடுக்க வேண்டும்
வெளியே எவ்வளவு குளிராக குளிராக இருக்கிறது
இந்த இந்த இந்த இந்த இந்த இந்த விதிமுறைகள் இந்த
குழந்தைகள் பெண்களுக்கு பெண்களுக்கு வயதிற்குக் ஆனால் ஆனால் ஆனால் ஆனால் ஆனால் ஆனால் ஆனால் ஆனால் ஆனால் ஆனால் ஆனால் ஆனால் ஆனால் தடுப்பூசி இல்லை
ஒரு நோய் உணவு உணவு மற்றும் மற்றும் மற்றும் நோய் நோய் வைரஸ் வைரஸ் நோய் நோய் ஆகும்
பட்டியலை சரிபார்க்கவும்
அறிகுறி இதயம் மனஇறுக்கம் நோயாளியின் உடல் தசைநார்கள்
இரத்தம் தூக்கும் அல்லது அல்லது கழிக்கும் கழிக்கும் அல்லது கழிக்கும் கழிக்கும் அல்லது
இந்த கன்னடம் மற்றும் மற்றும் குறிப்பாகத் மற்றும் இந்த
அவர் சில ஒரு இந்த ஒரு ஒரு ஒரு
இதன் விளைவாக முகலாயர்கள் மற்றும் ராஜ்கமல் வருகைக்கு வருகைக்கு மற்றும் மேலும் அதன் இருந்தது
லாகு என்பது குடிசைத்தொழில் ஒரு வடிவமாக இருக்கிறது
ஒரு பாலைவனத்தின் மற்றும் மேற்கொண்டு மற்றும் மற்றும் மற்றும் மற்றும் ஒரு
திவ்ய கினி மொழி மொழி மிகவும் நிறுவனம் நிறுவனம் இருக்கிறது
ஒரு வறையறு என்றால் என்ன
இன்று பிறந்தநாள் ந

In [ ]:
import csv

# Create a list of dictionaries where each dictionary represents a row
data = [{'valid_id': valid_id[i], 'tamil_pred': tamil_pred[i]} for i in range(len(valid_id))]

# Specify the CSV file path
csv_file_path = '/gdrive/MyDrive/tamil3.csv'

# Define the column names
fields = ['valid_id', 'tamil_pred']

# Write the data to the CSV file
with open(csv_file_path, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fields)
    writer.writeheader()
    writer.writerows(data)

print(f'Saved predictions to {csv_file_path}')

Saved predictions to /gdrive/MyDrive/tamil3.csv


In [ ]:
print(len(tamil_pred))
print(len(test_source))

16675
16675
